In [54]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, glob, os, warnings
from cv2 import blur as box_filter
warnings.filterwarnings("ignore")

## Dark Channel
Paper Reference: <b>Single Image Haze Removal Using Dark Channel Prior</b> by Kaiming He, Jian Sun et. al.

In [55]:
def plot(img, title, cmap=''):
    """
    Draw plot of a image with title passed as parameters 
    """
    if (len(cmap) != 0):
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(img)
    plt.title(title)
    plt.show()

In [56]:
def get_min_value_patch_channel(img, rows, columns, patch_size):
    """
    Get channelwise min value for each pixel
    """
    min_channel, border = np.zeros((rows, columns)), patch_size // 2
    for row in range(rows):
        for col in range(columns):
            min_channel[row - border][col - border] = np.min(img[row, col, :])
    return min_channel


def get_min_value_patch_whole(img, channel, rows, columns, patch_size):
    """
    Get dark channel image of a given image
    """
    dark_channel, border = np.zeros((img.shape[0], img.shape[1])), patch_size // 2
    
    # Applying Min Filter
    for row in range(border, rows - border):
        for col in range(border, columns - border):
            dark_channel[row - border][col - border] = np.min(
                channel[row - border:row + border, col - border:col + border])
    return dark_channel


def calculate_dark_channel_image(img, patch_size):
    """
    Calls above function after padding to generate dark channel image of a hazy image
    given as parameter.
    """
    dark_channel, border = np.zeros((img.shape[0], img.shape[1])), patch_size // 2
    white_pixel_value = [255, 255, 255]
    
    # padding rows and columns equal to half patch size to be used by Min Filter later 
    img = cv2.copyMakeBorder(img, border, border, border, border, cv2.BORDER_CONSTANT, value = white_pixel_value)
    no_rows, no_cols = img.shape[0], img.shape[1]
    
    # channel-wise min value
    min_channel = get_min_value_patch_channel(img, no_rows, no_cols, patch_size)
    
    # min pixel value in patch applied on each pixel (Min filtering)
    dark_channel = get_min_value_patch_whole(dark_channel, min_channel, no_rows, no_cols, patch_size)
    return dark_channel

### Atmospheric light - A

In [57]:
def atmospheric_light_A(h_img, dark_img):
    """
    Calculates Global Atmospheric light in hazy image using its dark channel image   
    """
    print('Image size:', dark_img.shape)
    img = h_img.copy()
    total_pixels = dark_img.size
    print('# pixels:', total_pixels)
    bright_pixels_count = total_pixels // 1000  # 0.1% from paper
    print('# bright pixels:', bright_pixels_count)
    
    # getting indices of brightest pixels in the deep-channel image and Indices sorted based on 
    # corresponding pixel values in Descending order
    sorted_indices_of_bright_pixels = np.argsort(dark_img, axis=None)[::-1]
    
    # top 0.1% =3296
    sorted_indices_of_bright_pixels = np.unravel_index(sorted_indices_of_bright_pixels[0:bright_pixels_count], dark_img.shape)
    print('Top 0.1% pixel indices:', sorted_indices_of_bright_pixels)
    
    # identify pixels in hazy image at the same indices as that of top 0.1% brightest pixels in its dark channel image  
    bright_pixels = img[sorted_indices_of_bright_pixels]  
    
    # first find intensities of selected bright pixels in the hazy image, 
    # and then choose the pixel with maximum intensity among them as global atmospheric light i.e. A
    A = bright_pixels[np.argmax(np.average(bright_pixels, axis=1))]
    return A

## Guided Filter
Paper Reference: <b>Guided Image Filtering</b> by Kaiming He, Jian Sun et. al.

In [58]:
def filtering(img, unrefined, r):
    """
    Calculates and returns hazy image and unrefined transmission map after box filtering them individually 
    and then filtering their product  
    """
    return box_filter(img, (r, r)), box_filter(unrefined, (r, r)), box_filter(img * unrefined, (r, r))

def guided_filter(h_img, unrefined_transmission_map, r, e):
    """
    This fuction takes unrefined transmission map as input image and applies guided filter on it 
    to refine it using the hazy image as guidance image
    """
    filtered_guidance_mean, filtered_transmission_mean, filtered_guided_transmission = filtering(h_img, unrefined_transmission_map, r)

    # calculating window parameters a and b
    a = filtered_guided_transmission - filtered_guidance_mean * filtered_transmission_mean
    filtered_guidance_variance = box_filter(h_img * h_img, (r, r)) - (filtered_guidance_mean * filtered_guidance_mean)
    a = a / (filtered_guidance_variance + e)
    b = filtered_transmission_mean - a * filtered_guidance_mean

    # deriving a_bar and b_bar using box filter on a and b respectively
    a_bar = box_filter(a, (r, r))
    b_bar = box_filter(b, (r, r))
    q = a_bar * h_img + b_bar  # q is the refined transmission map
    return q

### Dehazed Image

In [59]:
def Dehazed_image(h_img, transmissionMap, A, t0):
    """
    This function calculates Dehazed image by fitting values of A and transmission map
    in Haze imaging model equation
    """
    temp = np.copy(transmissionMap)
    temp[temp < t0] = t0  # clipping the values --> max(t1, t0)
    dehazed_img = np.zeros((h_img.shape))  # reconstruction img from formula
    channels = 3
    
    # Applying Haze imaging model channelwise
    for ch in range(channels):
        dehazed_img[:, :, ch] = ((h_img[:, :, ch] - A[ch]) / temp) + A[ch]
    return dehazed_img

## Simplest Color Balanced Image
Paper Reference: <b>Simplest Color Balance</b> by Nicolas Limare et.al

In [60]:
def color_balance(h_img, s):
    """
    Applying simple color balancing technique to refine contrast of image. The dehazed image obtained
    using above functions looks dim due to haze component removal. This function increases image sharpness
    """
    color_balanced_img = np.copy(h_img)
    max_intensity_levels, channels, V_min, V_max = 256, 3, 0, 255
    histogram = np.zeros((max_intensity_levels, 1))
    total_pixels = h_img.shape[0] * h_img.shape[1]
    for i in range(channels):
        color_channel = h_img[:, :, i]
        for p in range(max_intensity_levels):
            histogram[p] = np.sum((color_channel == p))
        for p in range(max_intensity_levels):
            histogram[p] = histogram[p - 1] + histogram[p]

        while (V_min < 255 and histogram[V_min] <= total_pixels * s):
            V_min += 1
        while (V_max > 0 and histogram[V_max] > total_pixels * (1 - s)):
            V_max -= 1
        color_channel[color_channel < V_min] = V_min
        color_channel[color_channel > V_max] = V_max
        color_balanced_img[:, :, i] = cv2.normalize(color_channel, color_channel.copy(), 0, 255, cv2.NORM_MINMAX)
    return color_balanced_img

In [61]:
def get_final_dehazed_image(hazy_img, patch_size):
    # get dark channel img of the given hazy image 
    dark_channel_img = calculate_dark_channel_image(hazy_img, patch_size).astype('uint8')
    
    # get global atmospheric light 
    A = atmospheric_light_A(hazy_img, dark_channel_img)
    
    # get unrefined transmission map
    OMEGA = 0.85  # min haze factor
    unrefined_transmission_map = 1 - (OMEGA * calculate_dark_channel_image(hazy_img / A, patch_size))
    plot(unrefined_transmission_map, 'UnRefined Transmission map', cmap='gray')
    
    # apply guided filter on the unrefined transmission map
    radius, epsilon = 30, 0.0001
    gray_scale_hazy_img = cv2.cvtColor(hazy_img, cv2.COLOR_BGR2GRAY) / 255
    transmission_map = guided_filter(gray_scale_hazy_img, unrefined_transmission_map, radius, epsilon)
    plot(transmission_map, 'Refined Transmission Map image', cmap='gray')
    
    # Haze imaging model
    haz_img = hazy_img.astype("double")
    dehaz_image = Dehazed_image(haz_img, transmission_map, A, 0.1)
    dehaz_image = ((dehaz_image - np.min(dehaz_image)) / (np.max(dehaz_image) - np.min(dehaz_image))) * 255
    
    # Apply color balancing in the dehazed image obtained
    s = 0.005
    finalImage = color_balance(np.uint8(dehaz_image), s)
    
    return dehaz_image, finalImage

In [62]:
def display_images(gr_truth_img, hazy_img, dehaz_image, finalImage):
    plot(gr_truth_img, 'Original image')
    plot(hazy_img, 'Hazy image')
    plot((np.uint8(dehaz_image)), 'Dehazed image')
    plot((np.uint8(finalImage)), 'Color balanced image')
    return

## Calling above functions stepwise for haze removal

In [63]:
# gt_path = r".\Dataset\I-HAZE\I-HAZY NTIRE 2018\GT/02_indoor_GT.jpg"
# hazy_img_path = r".\Dataset\I-HAZE\I-HAZY NTIRE 2018\hazy/02_indoor_hazy.jpg"

In [64]:
gt_path = r".\Dataset\O-HAZE\O-HAZY NTIRE 2018\GT/41_outdoor_GT.jpg"
hazy_img_path = r".\Dataset\O-HAZE\O-HAZY NTIRE 2018\hazy/41_outdoor_hazy.jpg"

In [65]:
gr_truth_img = cv2.imread(gt_path)
gr_truth_img = cv2.resize(gr_truth_img, (0, 0), fx=0.5, fy=0.5)
hazy_img = cv2.imread(hazy_img_path)
hazy_img = cv2.resize(hazy_img, (0, 0), fx=0.5, fy=0.5)

In [ ]:
patch_size = 30
dehaz_image, finalImage = get_final_dehazed_image(hazy_img, patch_size)

In [ ]:
display_images(gr_truth_img, hazy_img, dehaz_image, finalImage)